<a href="https://colab.research.google.com/github/AmirJlr/NLP-FUM/blob/master/Hands%20On/perisan_tweets_fast_clustering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install tweet-preprocessor
!pip install pandas
!pip install numpy
!pip install pytorch
!pip install hazm
!pip install sentence_transformers
!pip install transformers


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
  error: subprocess-exited-with-error
  
  × python setup.py bdist_wheel did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  ERROR: Failed building wheel for pytorch
  Running setup.py clean for pytorch
Failed to build pytorch
ERROR: Could not build wheels for pytorch, which is required to install pyproject.toml-based projects
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━

In [2]:
import re
import csv
from tqdm import tqdm
import pandas as pd
import numpy as np
import torch

import preprocessor as p
from hazm import *
from sentence_transformers.util import community_detection
from transformers import AutoConfig, AutoTokenizer, AutoModel


In [3]:
tweets = pd.read_csv('tweets.csv')

<ipython-input-3-8e525233028a>:1: DtypeWarning: Columns (9) have mixed types. Specify dtype option on import or set low_memory=False.
  tweets = pd.read_csv('tweets.csv')


In [5]:
tweets.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 84210 entries, 0 to 84209
Data columns (total 34 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   id               84210 non-null  int64  
 1   conversation_id  84210 non-null  int64  
 2   created_at       84210 non-null  int64  
 3   date             84210 non-null  object 
 4   time             84210 non-null  object 
 5   timezone         84210 non-null  object 
 6   user_id          84210 non-null  int64  
 7   username         84210 non-null  object 
 8   name             84210 non-null  object 
 9   place            42 non-null     object 
 10  tweet            84210 non-null  object 
 11  mentions         84210 non-null  object 
 12  urls             84210 non-null  object 
 13  photos           84210 non-null  object 
 14  replies_count    84210 non-null  int64  
 15  retweets_count   84210 non-null  int64  
 16  likes_count      84210 non-null  int64  
 17  hashtags    

In [6]:
tweets = tweets['tweet']

In [14]:
tweets = pd.DataFrame(tweets)

In [15]:
normalizer = Normalizer()


emoji_pattern = re.compile("["
                u"\U0001F600-\U0001F64F"  # emotions
                u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                u"\U0001F680-\U0001F6FF"  # transport & map symbols
                u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                u"\U00002702-\U000027B0"
                u"\U000024C2-\U0001F251"
                u"\U0001f926-\U0001f937"
                u'\U00010000-\U0010ffff'
                u"\u200d"
                u"\u2640-\u2642"
                u"\u2600-\u2B55"
                u"\u23cf"
                u"\u23e9"
                u"\u231a"
                u"\u3030"
                u"\ufe0f"
    "]+", flags=re.UNICODE)


characters = re.compile("""[A-Za-z0-9~`!#$%\^&*+=\-\[\]\\;,)(/{}|\\:<>\?«»"']+""")


def cleanTweet(tweet):
    p.set_options(p.OPT.URL, p.OPT.MENTION)
    tweet = p.clean(tweet).strip() #remove URLs and Mentions
    tweet = re.sub(characters, '', tweet)
    tweet = re.sub(emoji_pattern,'',tweet)
    tweet = normalizer.normalize(tweet)
    if len(tweet) > 10:
      return tweet



In [16]:
tweets['tweet'] = tweets['tweet'].apply(cleanTweet)

In [17]:
tweets = tweets[tweets['tweet'].astype(bool)].dropna().sample(frac=1).reset_index(drop=True)

In [18]:
tweets

,tweet
0,بازدید خجسته نماینده مردم همدان و فامنین از رو...
1,شبکه‌های اجتماعی تایید شده آقا آپارات کلوب آی....
2,با سلام بهتر نیست بجای تهمت زدن و فحش دادن قدر...
3,به نظر میرسد چین در عمل به تعهدات بین المللی و...
4,اصولا قراردادهای پنهانی زیاد داریم
...,...
75462,این رو ببینید
75463,سلام. کد رهگیری رو به بنده بفرمایید تا پی‌گیری...
75464,به راستی ما چرا نباید خدا را دوست داشته باشیم؟...
75465,روحانی آمد، پنجره گشوده شد مقاله‌ای از خاویرسو...


In [19]:
config = AutoConfig.from_pretrained("HooshvareLab/bert-base-parsbert-uncased")
tokenizer = AutoTokenizer.from_pretrained("HooshvareLab/bert-base-parsbert-uncased")
model = AutoModel.from_pretrained("HooshvareLab/bert-base-parsbert-uncased")


Some weights of the model checkpoint at HooshvareLab/bert-base-parsbert-uncased were not used when initializing BertModel: ['cls.predictions.transform.dense.weight', 'cls.predictions.decoder.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [20]:
input_ids = torch.tensor(tokenizer.encode(tweets['tweet'][0])).unsqueeze(0)
input_ids

tensor([[    2,  4401, 19885,  3408,  2342,  6459,   331, 40569,  2036,  3199,
          2464,   331,  4633,  6740,  7499,  5706,   300, 18902,  2417,   331,
          5576,  7744,  2665,  3762,  6459,     4]])

In [21]:
outputs = model(input_ids)
outputs

BaseModelOutputWithPoolingAndCrossAttentions(last_hidden_state=tensor([[[-0.0865, -0.9013,  0.5299,  ...,  0.6899,  0.2180,  0.9308],
         [ 0.9474, -0.7910, -0.5494,  ...,  1.3866,  0.8741,  0.2430],
         [ 1.1079, -0.7618, -0.2867,  ..., -0.1978, -0.1432, -0.6105],
         ...,
         [-0.3745, -0.6125,  0.1295,  ...,  0.7450,  0.2640, -0.6132],
         [ 0.2825, -1.3769, -0.9582,  ..., -0.3565,  0.8167, -0.3744],
         [-0.0573, -0.5212,  0.2241,  ..., -0.2086,  0.9356,  0.4763]]],
       grad_fn=<NativeLayerNormBackward0>), pooler_output=tensor([[-4.6270e-02,  9.8420e-01, -1.2810e-01, -9.4398e-01,  5.8141e-01,
          3.5256e-01, -5.2982e-01,  5.8072e-01, -9.1897e-01, -4.5752e-01,
          1.2227e-01, -2.0251e-01,  3.9313e-01,  9.0233e-01, -1.3611e-01,
          9.3963e-01,  5.9331e-01, -9.4047e-01, -1.7052e-01, -7.3841e-01,
          5.4528e-01,  2.2312e-01, -3.3384e-01, -2.2475e-01, -1.4778e-02,
          3.2643e-01,  3.0043e-01, -4.3935e-01, -2.2560e-01, -6.940

In [22]:
outputs = outputs['pooler_output']
outputs

tensor([[-4.6270e-02,  9.8420e-01, -1.2810e-01, -9.4398e-01,  5.8141e-01,
          3.5256e-01, -5.2982e-01,  5.8072e-01, -9.1897e-01, -4.5752e-01,
          1.2227e-01, -2.0251e-01,  3.9313e-01,  9.0233e-01, -1.3611e-01,
          9.3963e-01,  5.9331e-01, -9.4047e-01, -1.7052e-01, -7.3841e-01,
          5.4528e-01,  2.2312e-01, -3.3384e-01, -2.2475e-01, -1.4778e-02,
          3.2643e-01,  3.0043e-01, -4.3935e-01, -2.2560e-01, -6.9408e-01,
          4.6625e-01, -5.5230e-01,  9.9450e-01, -2.1265e-01,  2.3921e-01,
         -9.9029e-01,  1.5168e-01, -7.7151e-01, -9.1881e-01, -9.8763e-01,
          8.0951e-01,  8.0515e-02,  9.9652e-01,  1.5799e-01,  8.8324e-01,
         -9.9939e-01, -4.0825e-01,  1.5122e-02,  5.0213e-01,  4.0752e-01,
          2.1643e-01,  4.0186e-01,  8.2322e-02, -2.5197e-01,  1.1423e-01,
          6.1749e-01,  6.8855e-02,  4.5035e-01, -1.5229e-01, -1.2188e-01,
          4.5603e-01,  3.2849e-01,  5.3136e-01, -5.6323e-01, -3.8429e-01,
          1.5047e-01, -1.4028e-01,  8.

In [23]:
outputs.shape

torch.Size([1, 768])

In [24]:

with open('embeddings','wb') as file:
  batch_size = 50
  for batch in tqdm(range(int(len(tweets['tweet'])/batch_size)+1)):
    embeddings = []
    start = batch*batch_size
    end = (batch+1)*batch_size
    if end > len(tweets['tweet']):
      end = len(tweets['tweet'])
    for tweet in tweets['tweet'][start:end]:
      input_ids = torch.tensor(tokenizer.encode(tweet)).unsqueeze(0)
      outputs = model(input_ids)
      embeddings.append(outputs['pooler_output'].detach().numpy().squeeze())
    np.savetxt(file, embeddings)


  6%|▌         | 91/1510 [11:09<2:54:04,  7.36s/it]


KeyboardInterrupt: ignored

In [ ]:
embeddings = np.loadtxt('embeddings')
len(embeddings)

In [ ]:
clusters = community_detection(embeddings, threshold=0.9, init_max_size=len(embeddings))

In [ ]:
def write_to_csv_file(rows, OUTPUT_PATH):
    with open(OUTPUT_PATH, mode="w") as fp:
        fieldnames = [
            "cluster_id", 
            "tweet"]
        writer = csv.DictWriter(fp, fieldnames=fieldnames)
        writer.writeheader()
        for row in rows:
            writer.writerow(row)

In [ ]:
def collect_data(ds, clusters):
    rows = []
    for i, tweet_ids in enumerate(clusters):
        cluster_rows = []
        for tweet_id in tweet_ids:
            cluster_rows.append(
                {
                    "cluster_id": str(i + 1),
                    "tweet": ds["tweet"][tweet_id],
                }
            )
        cluster_rows = map(dict, set(tuple(sorted(d.items())) for d in cluster_rows))
        rows.extend(cluster_rows)
    return rows

In [ ]:
rows = collect_data(tweets, clusters)

In [ ]:
write_to_csv_file(rows, 'tweets_clustered.csv')

In [ ]:
clusters = pd.read_csv("tweets_clustered.csv")

for cluster_id in clusters['cluster_id'].unique():
    print(f"Cluster-{cluster_id}")
    for tweet in clusters[clusters['cluster_id']==cluster_id]['tweet'][:10]:
        print(tweet)
    print("\n")